In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from localenv import api_key
c = Census(api_key, year=2013)

In [3]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

census_data

[{'B01002_001E': 38.1,
  'B01003_001E': 4799277.0,
  'B17001_002E': 870631.0,
  'B19013_001E': 43253.0,
  'B19301_001E': 23680.0,
  'B23025_005E': 241930.0,
  'NAME': 'Alabama',
  'state': '01'},
 {'B01002_001E': 33.6,
  'B01003_001E': 720316.0,
  'B17001_002E': 69514.0,
  'B19013_001E': 70760.0,
  'B19301_001E': 32651.0,
  'B23025_005E': 32939.0,
  'NAME': 'Alaska',
  'state': '02'},
 {'B01002_001E': 36.3,
  'B01003_001E': 6479703.0,
  'B17001_002E': 1131901.0,
  'B19013_001E': 49774.0,
  'B19301_001E': 25358.0,
  'B23025_005E': 316360.0,
  'NAME': 'Arizona',
  'state': '04'},
 {'B01002_001E': 37.5,
  'B01003_001E': 2933369.0,
  'B17001_002E': 547328.0,
  'B19013_001E': 40768.0,
  'B19301_001E': 22170.0,
  'B23025_005E': 121235.0,
  'NAME': 'Arkansas',
  'state': '05'},
 {'B01002_001E': 35.4,
  'B01003_001E': 37659181.0,
  'B17001_002E': 5885417.0,
  'B19013_001E': 61094.0,
  'B19301_001E': 29527.0,
  'B23025_005E': 2168665.0,
  'NAME': 'California',
  'state': '06'},
 {'B01002_001E':

In [5]:
census_pd = pd.DataFrame(census_data)
census_pd.head()

,B01002_001E,B01003_001E,B17001_002E,B19013_001E,B19301_001E,B23025_005E,NAME,state
0,38.1,4799277.0,870631.0,43253.0,23680.0,241930.0,Alabama,01
1,33.6,720316.0,69514.0,70760.0,32651.0,32939.0,Alaska,02
2,36.3,6479703.0,1131901.0,49774.0,25358.0,316360.0,Arizona,04
3,37.5,2933369.0,547328.0,40768.0,22170.0,121235.0,Arkansas,05
4,35.4,37659181.0,5885417.0,61094.0,29527.0,2168665.0,California,06


In [4]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()

,State,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate
0,01,Alabama,4799277.0,38.1,43253.0,23680.0,870631.0,18.140878,5.040968
1,02,Alaska,720316.0,33.6,70760.0,32651.0,69514.0,9.650487,4.572854
2,04,Arizona,6479703.0,36.3,49774.0,25358.0,1131901.0,17.468409,4.882323
3,05,Arkansas,2933369.0,37.5,40768.0,22170.0,547328.0,18.658682,4.132961
4,06,California,37659181.0,35.4,61094.0,29527.0,5885417.0,15.628107,5.758662


In [5]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)